In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-pt12600f
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-pt12600f
  fatal: unable to access 'https://github.com/facebookresearch/segment-anything.git/': Failed to connect to github.com port 443 after 134929 ms: Connection timed out
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-pt12600f did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-pt12600f did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a 

In [2]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.utils.onnx import SamOnnxModel

import onnxruntime
from onnxruntime.quantization import QuantType
from onnxruntime.quantization.quantize import quantize_dynamic

In [3]:
onnx_model_path = "sam_onnx_vit_b.onnx"
checkpoint = "tmp/sam_vit_b_01ec64.pth"
model_type = "vit_b"
sam = sam_model_registry[model_type](checkpoint=checkpoint)

In [4]:
ort_session = onnxruntime.InferenceSession(onnx_model_path)

In [6]:
image = cv2.imread('sample/left_images/2018-10-19-10-33-08_2018-10-19-11-06-02-333.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
sam.to(device='cuda')
predictor = SamPredictor(sam)

In [7]:
predictor.set_image(image)
image_embedding = predictor.get_image_embedding().cpu().numpy()
image_embedding.shape

(1, 256, 64, 64)

In [ ]:
ort_inputs = {
    "image_embeddings": image_embedding,
    "point_coords": onnx_coord,
    "point_labels": onnx_label,
    "mask_input": onnx_mask_input,
    "has_mask_input": onnx_has_mask_input,
    "orig_im_size": np.array(image.shape[:2], dtype=np.float32)
}